
## for race map -- data 

In [1]:
import pandas as pd

import numpy as np

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
crime = pd.read_csv("Table_13_by_state_to_csv.csv")

df = crime


In [3]:

df = df.drop(df.index[0:5])


In [4]:
df = df.rename(columns = {
'Table 13':"state",
"Unnamed: 1":"agency_type" , 
"Unnamed: 2":"agency_name" , 
"Unnamed: 3":"race" , 
"Unnamed: 4":"religion"  ,  
"Unnamed: 5":"sexual_orientation",
"Unnamed: 6":"disability",
"Unnamed: 7":"gender",
"Unnamed: 8":"genderID",
"Unnamed: 9":"first_q",
"Unnamed: 10":"second_q",
"Unnamed: 11":"third_q",
"Unnamed: 12":"fourth_q",
"Unnamed: 13":"population",
    
})


In [5]:
del df["first_q"]
del df["second_q"]
del df["third_q"]
del df["fourth_q"]



In [6]:

df = df[:-7]



In [7]:
x = df


In [8]:
#fill in these two string columns

x[['state', 'agency_type']] = x[['state','agency_type']].fillna(method='ffill')


get rid of rows with NAN


In [9]:
x = x.dropna()


In [10]:
g = x[x['agency_type'] == "Cities"]


In [11]:
#df[pd.notnull(df['c'])]
h = g[pd.notnull(g['agency_name'])]


In [12]:
t = h


In [13]:
t['population'] = t['population'].str.replace(',', '')


In [14]:
#t.iloc[:, :].apply(pd.to_numeric)  #all rows, column 1-6
t = t.apply(pd.to_numeric, errors='ignore')



In [15]:
u = t

u["race_per_100000"] = u['race']/u['population'] * 100000
u["religion_per_100000"] = (u['religion']/u['population']) * 100000
u["SO_per_100000"] = (u['sexual_orientation']/u['population']) * 100000
u["disability_per_100000"] = (u['disability']/u['population']) * 100000
u["gender_per_100000"] = (u['gender']/u['population']) * 100000
u["genderID_per_100000"] = (u['genderID']/u['population']) * 100000


u = u.round(decimals=2)

# city population size 

In [16]:
w = u.loc[u['population'] > 50000]


In [17]:
#fixes
w = w.replace("Boston3", "Boston")
w = w.replace("Greensboro3", "Greensboro")
w = w.replace("Toledo3", "Toledo")


In [18]:
w = w.replace("Las Vegas Metropolitan Police Department3", "Las Vegas")
w = w.replace("Hartford3", "Hartford")
w = w.replace("Overland Park3", "Overland Park")
w = w.replace("Newton3", "Newton")
w = w.replace("Quincy3", "Quincy")
w = w.replace("Warren3", "Warren")
w = w.replace("Spokane3", "Spokane")
w = w.replace("Bloomington3", "Bloomington")
w = w.replace("Eugene3", "Eugene")
w = w.replace("Somerville3", "Somerville")


In [19]:
w = w.replace("Nashville Metropolitan", "Nashville")
w = w.replace("Louisville Metro", "Louisville")



## getting the lat lon coordiantes

#cut and paste this file into atom where.js
#go to terminal and switch to python2
#run geoload.py to connect with google map API
#the  file with the lat and lon is where.js

this sheet to use google search on long/lat ...google return will mess up formatting of words though ie st louis not st. louis

In [20]:
w["search"] = w["agency_name"].map(str) + " " + w["state"]
search = w["search"]

#search.to_csv("city_longitude_search1", index=False)


In [21]:
formatLatLon = pd.read_csv("formatted_lat_lon")

In [22]:
latFormat = formatLatLon


In [23]:
latFormat["city, state"] = latFormat["City"].map(str) + ", " + latFormat["fullname"]



In [24]:
del latFormat["City"]
del latFormat["state"]
del latFormat["fullname"]

In [25]:
latFormat.isnull().sum()

lat            0
lon            0
city, state    0
dtype: int64

## FORMAT THESe  so that  null city names match w2 city names -- some whitespaces
should have been fixed in google maps..oh well..failure to strip white space after first word and before comma in google maps

In [26]:
w['agency_name'] = w['agency_name'].str.replace('Township','')
w['agency_name'] = w['agency_name'].str.replace('Town','')
w['agency_name'] = w['agency_name'].str.strip(' ')
w['agency_name'] = w['agency_name'].str.replace('Charlotte-Mecklenburg','Charlotte')
w['agency_name'] = w['agency_name'].str.replace('Athens-Clarke County','Athens')
w['agency_name'] = w['agency_name'].str.replace('Parsippany Troy Hills','Parsippany-Troy Hills')
w['agency_name'] = w['agency_name'].str.replace('Mount Vernon','Mt Vernon')
w['agency_name'] = w['agency_name'].str.replace('Hamilton , Mercer County','Hamilton')
w['agency_name'] = w['agency_name'].str.replace('Port St. Lucie' , 'Port St Lucie')
w['agency_name'] = w['agency_name'].str.replace('Shelby', 'Shelby ')
w['agency_name'] = w['agency_name'].str.replace('Bloomfield' , 'Bloomfield ')
w['agency_name'] = w['agency_name'].str.replace('Franklin , Somerset County' , 'Franklin')
w['agency_name'] = w['agency_name'].str.replace('Piscataway' , 'Piscataway ')  
w['agency_name'] = w['agency_name'].str.replace('Woodbridge' , 'Woodbridge ')
w['agency_name'] = w['agency_name'].str.replace('West Chester' , 'West Chester ')
  


latFormat["city, state"] = latFormat["city, state"].str.replace('Lakewood , New Jersey' , 'Lakewood, New Jersey')

latFormat["city, state"] = latFormat["city, state"].str.replace('Mt Pleasant' , 'Mount Pleasant')

latFormat["city, state"] = latFormat["city, state"].str.replace('Franklin ' , 'Franklin')

latFormat["city, state"] = latFormat["city, state"].str.replace('Rocky Mt. ' , 'Rocky Mount')

latFormat["city, state"] = latFormat["city, state"].str.replace('Hamilton ' , 'Hamilton')


In [27]:
latFormat[latFormat["city, state"].str.contains("York")]#didn't strip whitespac in google or county in hate crime df
w[w["agency_name"].str.contains("York")]#missing in gogole map set

,state,agency_type,agency_name,race,religion,sexual_orientation,disability,gender,genderID,population,race_per_100000,religion_per_100000,SO_per_100000,disability_per_100000,gender_per_100000,genderID_per_100000,search
1209,New York,Cities,New York,71,154,81,1,0,0,8550861,0.83,1.8,0.95,0.01,0.0,0.0,New York New York


In [28]:
w["city, state"] = w["agency_name"].map(str) + ", " + w["state"]



In [29]:
w.isnull().sum()

state                    0
agency_type              0
agency_name              0
race                     0
religion                 0
sexual_orientation       0
disability               0
gender                   0
genderID                 0
population               0
race_per_100000          0
religion_per_100000      0
SO_per_100000            0
disability_per_100000    0
gender_per_100000        0
genderID_per_100000      0
search                   0
city, state              0
dtype: int64

In [30]:
merged_df = w.merge(latFormat, how='left', on='city, state')

add new york lat and lon to merged df  40.7128, -74.0059

make a column on both df's that concats city and state'

In [31]:
dupes = merged_df[merged_df.duplicated(['city, state'], keep=False)]
dupes

,state,agency_type,agency_name,race,religion,sexual_orientation,disability,gender,genderID,population,race_per_100000,religion_per_100000,SO_per_100000,disability_per_100000,gender_per_100000,genderID_per_100000,search,"city, state",lat,lon
258,Minnesota,Cities,St. Paul,10,1,6,0,0,0,300721,3.33,0.33,2.00,0.0,0.0,0.0,St. Paul Minnesota,"St. Paul, Minnesota",44.953703,-93.089958
259,Minnesota,Cities,St. Paul,10,1,6,0,0,0,300721,3.33,0.33,2.00,0.0,0.0,0.0,St. Paul Minnesota,"St. Paul, Minnesota",44.953703,-93.089958
273,Nevada,Cities,Las Vegas,22,4,15,0,0,0,1562134,1.41,0.26,0.96,0.0,0.0,0.0,Las Vegas Nevada,"Las Vegas, Nevada",36.102372,-115.174556
274,Nevada,Cities,Las Vegas,22,4,15,0,0,0,1562134,1.41,0.26,0.96,0.0,0.0,0.0,Las Vegas Nevada,"Las Vegas, Nevada",36.169941,-115.139830


## with new sheet may have to change indexes in fxns below!

In [32]:

merged_df = merged_df.drop(merged_df.index[[259,274]])

In [33]:
dupes2 = merged_df[merged_df.duplicated(['city, state'], keep=False)]
dupes2

,state,agency_type,agency_name,race,religion,sexual_orientation,disability,gender,genderID,population,race_per_100000,religion_per_100000,SO_per_100000,disability_per_100000,gender_per_100000,genderID_per_100000,search,"city, state",lat,lon


In [34]:
merged_df.isnull().sum()

state                    0
agency_type              0
agency_name              0
race                     0
religion                 0
sexual_orientation       0
disability               0
gender                   0
genderID                 0
population               0
race_per_100000          0
religion_per_100000      0
SO_per_100000            0
disability_per_100000    0
gender_per_100000        0
genderID_per_100000      0
search                   0
city, state              0
lat                      1
lon                      1
dtype: int64

In [35]:
#add new york lat lon
merged_df[merged_df["agency_name"].str.contains("York")]

,state,agency_type,agency_name,race,religion,sexual_orientation,disability,gender,genderID,population,race_per_100000,religion_per_100000,SO_per_100000,disability_per_100000,gender_per_100000,genderID_per_100000,search,"city, state",lat,lon
304,New York,Cities,New York,71,154,81,1,0,0,8550861,0.83,1.8,0.95,0.01,0.0,0.0,New York New York,"New York, New York",NaN,NaN


In [36]:
merged_df.set_value(304, 'lat', 40.7128)
merged_df.set_value(304, 'lon', -74.0059)

,state,agency_type,agency_name,race,religion,sexual_orientation,disability,gender,genderID,population,race_per_100000,religion_per_100000,SO_per_100000,disability_per_100000,gender_per_100000,genderID_per_100000,search,"city, state",lat,lon
0,Alabama,Cities,Birmingham,3,0,2,0,0,0,212291,1.41,0.00,0.94,0.00,0.00,0.00,Birmingham Alabama,"Birmingham, Alabama",33.520661,-86.802490
1,Alabama,Cities,Hoover,4,0,0,0,0,0,85163,4.70,0.00,0.00,0.00,0.00,0.00,Hoover Alabama,"Hoover, Alabama",33.405387,-86.811378
2,Alaska,Cities,Anchorage,4,0,1,0,0,0,301239,1.33,0.00,0.33,0.00,0.00,0.00,Anchorage Alaska,"Anchorage, Alaska",61.218056,-149.900278
3,Arizona,Cities,Avondale,1,1,0,0,0,0,80533,1.24,1.24,0.00,0.00,0.00,0.00,Avondale Arizona,"Avondale, Arizona",33.435598,-112.349602
4,Arizona,Cities,Chandler,1,0,0,0,0,0,258875,0.39,0.00,0.00,0.00,0.00,0.00,Chandler Arizona,"Chandler, Arizona",33.306160,-111.841250
5,Arizona,Cities,Gilbert,3,1,0,0,0,0,247324,1.21,0.40,0.00,0.00,0.00,0.00,Gilbert Arizona,"Gilbert, Arizona",33.352826,-111.789027
6,Arizona,Cities,Glendale,5,2,1,0,0,0,240374,2.08,0.83,0.42,0.00,0.00,0.00,Glendale Arizona,"Glendale, Arizona",33.538652,-112.185987
7,Arizona,Cities,Mesa,2,2,0,0,0,0,471034,0.42,0.42,0.00,0.00,0.00,0.00,Mesa Arizona,"Mesa, Arizona",33.415184,-111.831472
8,Arizona,Cities,Phoenix,137,40,49,5,0,0,1559744,8.78,2.56,3.14,0.32,0.00,0.00,Phoenix Arizona,"Phoenix, Arizona",33.448377,-112.074037
9,Arizona,Cities,Scottsdale,1,1,1,0,0,0,233872,0.43,0.43,0.43,0.00,0.00,0.00,Scottsdale Arizona,"Scottsdale, Arizona",33.494170,-111.926052


In [37]:
merged_df[merged_df["agency_name"].str.contains("York")]

,state,agency_type,agency_name,race,religion,sexual_orientation,disability,gender,genderID,population,race_per_100000,religion_per_100000,SO_per_100000,disability_per_100000,gender_per_100000,genderID_per_100000,search,"city, state",lat,lon
304,New York,Cities,New York,71,154,81,1,0,0,8550861,0.83,1.8,0.95,0.01,0.0,0.0,New York New York,"New York, New York",40.7128,-74.0059


In [38]:
merged_df.isnull().sum()

state                    0
agency_type              0
agency_name              0
race                     0
religion                 0
sexual_orientation       0
disability               0
gender                   0
genderID                 0
population               0
race_per_100000          0
religion_per_100000      0
SO_per_100000            0
disability_per_100000    0
gender_per_100000        0
genderID_per_100000      0
search                   0
city, state              0
lat                      0
lon                      0
dtype: int64

In [39]:
fixNull = merged_df[merged_df.isnull().any(axis=1)]
fix = fixNull["agency_name"]
list(fix)

[]

## making list for set of csvs to keep them smaller and make it easier for js programming data viz

In [40]:
col_list_num = ['race', 'religion',
       'sexual_orientation', 'disability', 'gender', 'genderID']

In [41]:
merged_df['total_num'] = merged_df[col_list_num].sum(axis=1)

In [42]:
col_list_per = ['race_per_100000', 'religion_per_100000', 'SO_per_100000',
       'disability_per_100000', 'gender_per_100000', 'genderID_per_100000']

In [43]:
merged_df['total_per'] = merged_df[col_list_per].sum(axis=1)

In [55]:
#this is the latest city one have to break them up so they render without fixing load..
raceCity = merged_df[["agency_name","lat","lon","race","race_per_100000",'population']]
religionCity = merged_df[["agency_name","lat","lon","religion","religion_per_100000",'population']]
soCity = merged_df[["agency_name","lat","lon","sexual_orientation","SO_per_100000",'population']]
disCity = merged_df[["agency_name","lat","lon","disability","disability_per_100000",'population']]
genderCity = merged_df[["agency_name","lat","lon","gender","gender_per_100000",'population']]
genderIDCity = merged_df[["agency_name","lat","lon","genderID","genderID_per_100000",'population']]

totalNumber = merged_df[["agency_name","lat","lon","total_num",'population']]

per100000 = merged_df[["agency_name","lat","lon", "total_num","total_per",'population']]


In [56]:
raceCity.rename(columns={'agency_name': 'city'}, inplace=True)
raceCity.rename(columns={'race': 'num'}, inplace=True)
raceCity.rename(columns={'race_per_100000': 'per_100000'}, inplace=True)

religionCity.rename(columns={'agency_name': 'city'}, inplace=True)
religionCity.rename(columns={'religion': 'num'}, inplace=True)
religionCity.rename(columns={'religion_per_100000': 'per_100000'}, inplace=True)


soCity.rename(columns={'agency_name': 'city'}, inplace=True)
soCity.rename(columns={'sexual_orientation': 'num'}, inplace=True)
soCity.rename(columns={'SO_per_100000': 'per_100000'}, inplace=True)

#disCity = new1[["agency_name","lat","lon","disability","disability_per_100000"]]
disCity.rename(columns={'agency_name': 'city'}, inplace=True)
disCity.rename(columns={'disability': 'num'}, inplace=True)
disCity.rename(columns={'disability_per_100000': 'per_100000'}, inplace=True)


#genderCity = [["agency_name","lat","lon","gender","gender_per_100000"]]
genderCity.rename(columns={'agency_name': 'city'}, inplace=True)
genderCity.rename(columns={'gender': 'num'}, inplace=True)
genderCity.rename(columns={'gender_per_100000': 'per_100000'}, inplace=True)

#genderIDCity = [["agency_name","lat","lon","genderID","genderID_per_100000"]]
genderIDCity.rename(columns={'agency_name': 'city'}, inplace=True)
genderIDCity.rename(columns={'genderID': 'num'}, inplace=True)
genderIDCity.rename(columns={'genderID_per_100000': 'per_100000'}, inplace=True)


#genderIDCity = [["agency_name","lat","lon","genderID","genderID_per_100000"]]
per100000.rename(columns={'agency_name': 'city'}, inplace=True)
#per100000.rename(columns={'': 'num'}, inplace=True)
per100000.rename(columns={'total_per': 'per_100000'}, inplace=True)

per100000.rename(columns={'total_num': 'num'}, inplace=True)


/Users/hw/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


## writing to csv's below

In [57]:
perAllCity = per100000.sort_values('per_100000', ascending=False)
perAllCity = per100000.loc[per100000['per_100000'] > 0]
perAllCity.head()

perAllCity.to_csv("perAllCity_2015", index=False)

In [47]:
raceCity = raceCity.sort_values('per_100000', ascending=False)
raceCity = raceCity.loc[raceCity['per_100000'] > 0]
raceCity.head()

#raceCity.to_csv("raceCity_2015", index=False)

,city,lat,lon,num,per_100000,population
329,Lakewood,41.481993,-81.798191,9,17.77,50653
220,Quincy,42.252877,-71.002270,16,17.08,93671
341,Eugene,44.052069,-123.086754,25,15.47,161608
326,Columbus,39.961176,-82.998794,114,13.45,847745
209,Boston,42.360083,-71.058880,81,12.18,665258


In [48]:
religionCity = religionCity.sort_values('per_100000', ascending=False)
religionCity = religionCity.loc[religionCity['per_100000'] > 0]
religionCity.head()

#religionCity.to_csv("religionCity_2015", index=False)

,city,lat,lon,num,per_100000,population
287,Lakewood,40.082129,-74.209701,17,17.76,95743
288,New Brunswick,40.486216,-74.451819,7,12.16,57580
415,Redmond,47.673988,-122.121512,5,8.26,60542
205,Portland,43.661471,-70.255326,5,7.48,66816
158,Miami Beach,25.790654,-80.130045,6,6.48,92641


In [49]:

soCity = soCity.sort_values('per_100000', ascending=False)
soCity = soCity.loc[soCity['per_100000'] > 0]
soCity.head()

#soCity.to_csv("soCity_2015", index=False)

,city,lat,lon,num,per_100000,population
288,New Brunswick,40.486216,-74.451819,5,8.68,57580
209,Boston,42.360083,-71.058880,46,6.91,665258
328,Hamilton,39.399501,-84.561335,4,6.40,62526
426,Charleston,38.349820,-81.632623,3,5.98,50176
283,Hoboken,40.743991,-74.032363,3,5.55,54050


In [50]:
disCity = disCity.sort_values('per_100000', ascending=False)
disCity = disCity.loc[disCity['per_100000'] > 0]
disCity.head()

#disCity.to_csv("disCity_2015", index=False)

,city,lat,lon,num,per_100000,population
232,Flint,43.012527,-83.687456,2,2.04,98221
426,Charleston,38.349820,-81.632623,1,1.99,50176
85,Placentia,33.871073,-117.862755,1,1.90,52753
283,Hoboken,40.743991,-74.032363,1,1.85,54050
340,Corvallis,44.564566,-123.262044,1,1.81,55100


In [51]:
genderCity = genderCity.sort_values('per_100000', ascending=False)
genderCity = genderCity.loc[genderCity['per_100000'] > 0]
genderCity.head()

#genderCity.to_csv("genderCity_2015", index=False)

,city,lat,lon,num,per_100000,population
223,Taunton,41.900101,-71.089767,3,5.29,56731
217,Medford,42.418430,-71.106164,2,3.46,57742
221,Somerville,42.387597,-71.099497,1,1.25,79734
408,Bellingham,48.751911,-122.478685,1,1.19,83976
218,Newton,42.337041,-71.209221,1,1.12,89053


In [52]:
genderIDCity = genderIDCity.sort_values('per_100000', ascending=False)
genderIDCity = genderIDCity.loc[genderIDCity['per_100000'] > 0]
genderIDCity.head()

#genderIDCity.to_csv("genderIDCity_2015", index=False)

,city,lat,lon,num,per_100000,population
223,Taunton,41.900101,-71.089767,10,17.63,56731
216,Malden,42.425096,-71.066163,5,8.17,61211
215,Lynn,42.466763,-70.949494,7,7.56,92584
213,Haverhill,42.776201,-71.077280,3,4.77,62895
224,Waltham,42.376485,-71.235611,3,4.72,63590


In [53]:
#totalNumber = new1[["agency_name","lat","lon","total_num"]]

totalNumber = totalNumber.sort_values('total_num', ascending=False)
totalNumber = totalNumber.loc[totalNumber['total_num'] > 0]
totalNumber.head()

#totalNumber.to_csv("totalNumber_2015", index=False)

,agency_name,lat,lon,total_num,population
304,New York,40.712800,-74.005900,307,8550861
8,Phoenix,33.448377,-112.074037,231,1559744
62,Los Angeles,34.052234,-118.243685,192,3962726
209,Boston,42.360083,-71.058880,148,665258
326,Columbus,39.961176,-82.998794,137,847745


In [54]:
#per100000 = new1[["agency_name","lat","lon","per_100000"]]


per100000 = per100000.sort_values('total_per', ascending=False)
per100000 = per100000.loc[per100000['total_per'] > 0]
per100000.head()

#per100000.to_csv("per100000_2015", index=False)

KeyError: 'total_per'